In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx

In [ ]:
def visualize_graph(n_nodes, positions, children):
    fig = go.Figure()

    # Add lines from parent to children
    for i in range(n_nodes):
        for child in children[i]:
            fig.add_trace(go.Scatter(
                x=[positions[i, 0], positions[child, 0]],
                y=[positions[i, 1], positions[child, 1]],
                mode='lines',
                line=dict(color='gray', width=2),
                showlegend=False
            ))

    # Add nodes on top
    scatter = go.Scatter(
        x=positions[:, 0], 
        y=positions[:, 1], 
        mode='markers',
        text=[str(i) for i in range(n_nodes)],
        textposition='middle center',
        marker=dict(size=5, color='lightblue', line=dict(color='black', width=2))
    )
    fig.add_trace(scatter)

    fig.update_layout(
        title="Random Tree Visualization",
        xaxis_title="X",
        yaxis_title="Y",
        showlegend=False
    )
    return fig

def animate_trajectories(fig, node_positions, trajectories, node_colors=None, highlight=None):        
    trajectories = np.asarray(trajectories)

    n_nodes = node_positions.shape[0]
    n_trajectories = trajectories.shape[0]
    n_frames = trajectories.shape[1]
    frames = []
    trace = len(fig.data)

    # Use provided colors or default to trajectory index
    if node_colors is None:
        node_colors = np.full([n_frames, n_nodes], 'rgba(0, 0, 0, 0)', dtype=object)
    else:
        node_colors = np.asarray(node_colors, dtype=object)

    n_colors = len(px.colors.qualitative.Set3)
    for i in range(n_trajectories):
        color = px.colors.qualitative.Set3[i % n_colors]
        node_colors[np.arange(n_frames), trajectories[i]] = color

    for i in range(n_frames):
        x = node_positions[:, 0]
        y = node_positions[:, 1]

        node_size = np.full([n_nodes], 8).astype(int)
        if highlight is not None:
            node_size[trajectories[highlight, i]] = 14
        
        frame_colors = node_colors[i]
        frames.append(
            go.Frame(
                data=go.Scatter(
                    x=x, y=y,
                    hoverinfo='none',
                    mode='markers',
                    marker=dict(size=node_size, color=frame_colors, opacity=1)
                ),
                traces=[trace]
            )
        )

    fig.add_trace(frames[0].data[0])
    fig.update(frames=frames)

    fig.update_layout(updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, {"frame": {"duration": 500, "redraw": True}, 
                                        "fromcurrent": True, 
                                        "transition": {"duration": 0}}])]
        )])

In [ ]:
class TreeNode:
    def __init__(self, value, parent=None):
        self.value = value
        self.parent = parent
        self.children = []
    
    def add_child(self, child_node):
        child_node.parent = self
        self.children.append(child_node)
    
    def __repr__(self):
        return f"Node({self.value})"

class ExploringAgent:
    def __init__(self, id, starting_node, true_map, root=0, all_agents=None):
        self.id = id
        self.root = root
        self.current_node = starting_node
        self.node_map = {}
        self.true_map = true_map
        self.all_agents = all_agents or []
        self.update_map()
        self.node_map[0].update({'responsibilities': all_agents.copy()})
    
    def visit_node(self, node):
        """Visit a node and update the map with its structure"""
        node = self.true_map[node]
        current_entry = self.node_map.get(node.value, {'visited': False})
        if not current_entry['visited']:
            current_entry.update({
                'parent': node.parent.value if node.parent else None,
                'children': [child.value for child in node.children],
                'visited': True
            })
            self.node_map[node.value] = current_entry
    
    def current_children(self):
        return self.node_map[self.current_node]['children']
    
    def current_parent(self):
        return self.node_map[self.current_node].get('parent', None)
    
    def assign_responsibilities(self):
        """
        Calculate responsibilities for children of current node and save them in node_map.
        Uses responsibilities from current node, or self.all_agents if current node has no parent.
        """
        # Get the list of agents to distribute
        if self.node_map[self.current_node]['parent'] is None:
            # Root node case: use all_agents
            agents = self.all_agents
        else:
            # Non-root node case: get responsibilities from current node
            agents = self.node_map[self.current_node]['responsibilities']
        
        # Calculate responsibilities for children
        responsibilities = self.calculate_responsibilities(self.current_node, agents)
        
        # Save responsibilities in node_map for each child
        for child_value in self.node_map[self.current_node]['children']:
            if child_value not in self.node_map:
                self.node_map[child_value] = {
                    'responsibilities': responsibilities[child_value],
                    'visited': False
                }
            else:
                # Update the child's responsibilities in the node_map
                self.node_map[child_value]['responsibilities'] = responsibilities[child_value]
    
    def _is_subtree_fully_explored(self, node_value):
        """
        Check if all nodes in the subtree rooted at node_value that are this agent's 
        responsibility have been visited.
        """
        if node_value not in self.node_map:
            return False
            
        node_info = self.node_map[node_value]
        
        # Check if this node is our responsibility and if it's been visited
        responsibilities = node_info.get('responsibilities', [])
        if self.id not in responsibilities:
            if self.id == 0:
                print('weird')
            return True

        if not node_info.get('visited', False):
            return False
        
        # Recursively check all children
        for child_value in node_info.get('children', []):
            if not self._is_subtree_fully_explored(child_value):
                return False
            
        if self.id == 0:
            print('default')
        return True
    
    def open_explore_step(self):
        """
        Execute one exploration step: assign responsibilities, then move to first unexplored 
        child that is this agent's responsibility, or move to parent if none found.
        """
        self.assign_responsibilities()

        for child_value in self.current_children():
            if not self._is_subtree_fully_explored(child_value):
                return child_value
        if not self._is_subtree_fully_explored(self.root):
            return self.current_parent()
        return None
    
    def update_map(self):
        self.visit_node(self.current_node)
    
    def calculate_responsibilities(self, node, agents):
        """
        Distribute agents across the children of a given node.
        
        Args:
            node: TreeNode whose children will be assigned to agents
            agents: List of ExploringAgent instances to distribute
        
        Returns:
            dict: Mapping of child nodes to lists of assigned agents
        """

        children = self.node_map[node].get('children', None)
        if not children:
            return {}
        
        responsibilities = {child: [] for child in children}
        n_agents = len(agents)
        n_children = len(children)

        if n_agents >= n_children:
            agents_per_child = n_agents // n_children
            extra_agents = n_agents % n_children

            remaining_agents = agents.copy()
            for i, child in enumerate(children):
                n_assigned = agents_per_child + (1 if i < extra_agents else 0)
                responsibilities[child].extend(remaining_agents[:n_assigned])
                remaining_agents = remaining_agents[n_assigned:]
        else:
            children_per_agent = n_children // n_agents
            extra_children = n_children % n_agents
            
            remaining_children = children.copy()
            for i, agent in enumerate(agents):
                n_assigned = children_per_agent + (1 if i < extra_children else 0)
                for child in remaining_children[:n_assigned]:
                    responsibilities[child].append(agent)
                remaining_children = remaining_children[n_assigned:]

        return responsibilities

In [ ]:
def synchronize_agent_maps(agents):
    """
    Synchronize the maps of multiple agents.
    Creates a unified map with all nodes from all agents,
    where a node is marked as visited if any agent has visited it.
    
    Args:
        agents: List of ExploringAgent instances
    """
    if len(agents) <= 1:
        return
    
    synchronized_map = {}
    
    for agent in agents:
        for node_value, info in agent.node_map.items():
            synced_entry = synchronized_map.get(node_value, info.copy())
            visited = synced_entry.get('visited', False) or info.get('visited', False)
            synced_entry.update(info)
            synced_entry['visited'] = visited
            synchronized_map[node_value] = synced_entry
    
    for agent in agents:
        agent.node_map = synchronized_map.copy()

In [ ]:
def find_groups(agents, edges):
    def is_connected(i, j):
        i = agents[i].current_node
        j = agents[j].current_node
        return (i == j) or np.all([i, j] == edges, axis=1).any() or np.all([j, i] == edges, axis=1).any()

    roots = np.arange(len(agents)).tolist()
    for i in range(0, len(agents)):
        for j in range(i+1, len(agents)):
            if is_connected(i, j):
                roots[j] = roots[i]

    groups = {}
    for agent, root in zip(agents, roots):
        if root in groups:
            groups[root].append(agent)
        else:
            groups[root] = [agent]
    return list(groups.values())

In [ ]:
def generate_random_tree(n_nodes=10):
    end = np.arange(n_nodes)
    start = (end * np.random.uniform(size=n_nodes)).astype(int)
    edges = np.stack([start, end], axis=1)[1:]
    
    children = [edges[edges[:, 0] == i, 1].tolist() for i in range(n_nodes)]
    
    # Create nodes
    nodes = {i: TreeNode(i) for i in range(n_nodes)}
    
    # Build tree structure
    for i, child_indices in enumerate(children):
        for child_idx in child_indices:
            nodes[i].add_child(nodes[child_idx])
    
    G = nx.DiGraph()
    for parent, local_children in enumerate(children):
        for child in local_children:
            G.add_edge(parent, child)
            
    positions = nx.nx_agraph.graphviz_layout(G, prog='dot')
    positions = np.asarray([value for key, value in sorted(positions.items())])

    return nodes, edges, positions, children

In [ ]:
n_nodes = 30
true_map, edges, positions, children = generate_random_tree(n_nodes=n_nodes)

In [ ]:
n_agents = 4
agents = [ExploringAgent(i, 0, true_map, root=0, all_agents=list(range(n_agents))) for i in range(n_agents)]
trajectories = [[0] for _ in range(n_agents)]
explored_nodes = [[[]] for _ in range(n_agents)]

for i in range(50):
    connected_agent_groups = find_groups(agents, edges)
    for group in connected_agent_groups:
        synchronize_agent_maps(group)
        no_collision_move(group, lambda agent: agent.open_explore_step())
        
    for agent in agents:
        agent.update_map()
        trajectories[agent.id].append(agent.current_node)
        
    for idx, agent in enumerate(agents):
        explored_nodes[idx].append([k for k, v in agent.node_map.items() if v.get('visited', False)])

In [ ]:
def get_node_color_for_agent(i):
    n_timesteps = len(explored_nodes[i])
    node_colors = np.full((n_timesteps, n_nodes), 'rgba(0, 0, 0, 255)', dtype=object)

    for t, explored_list in enumerate(explored_nodes[i]):
        for node_id in explored_list:
            node_colors[t, node_id] = 'rgba(100, 255, 100, 255)'
    return node_colors

In [ ]:
fig = visualize_graph(n_nodes, positions, children)
animate_trajectories(fig, positions, trajectories, node_colors=get_node_color_for_agent(1), highlight=1)

fig.show()

In [ ]:
def no_collision_move(agents, calc_next_node):
    target_nodes = {agent.id: calc_next_node(agent) for agent in agents}
    agents = sorted(agents, key=lambda agent: agent.id)
    current_nodes = {agent.id: agent.current_node for agent in agents}

    taken_targets = []
    remaining_agents = []
    inactive_positions = {}

    for agent in agents:
        target = target_nodes[agent.id]
        if target is None:
            inactive_positions[agent.current_node] = agent
            continue
        if target in taken_targets:
            continue
        remaining_agents.append(agent)
        taken_targets.append(target)
        
    
    for agent_iter in remaining_agents.copy():
        agent = agent_iter
        target = target_nodes[agent.id]
        while target in inactive_positions:
            swap_agent = inactive_positions[target]

            tmp = agent.id
            agent.id = swap_agent.id
            swap_agent.id = tmp

            new_target = calc_next_node(agent)           

            remaining_agents.remove(agent)
            inactive_positions[agent.current_node] = agent
            
            if new_target is None:
                break

            remaining_agents.append(swap_agent)
            del inactive_positions[target]

            agent = swap_agent
            target_nodes[agent.id] = new_target
            target = new_target

    moved_agents = []
    blocked_nodes = current_nodes.copy()

    n_remaining = -1
    while len(remaining_agents) != n_remaining:
        n_remaining = len(remaining_agents)
        for agent in remaining_agents.copy():
            if target_nodes[agent.id] not in blocked_nodes.values():
                moved_agents.append(agent)
                del blocked_nodes[agent.id]
                remaining_agents.remove(agent)
    
    for agent in moved_agents:
        agent.current_node = target_nodes[agent.id]